In [1]:
import os
import sys
import imgaug
import pickle
import sklearn

from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    roc_auc_score,
    balanced_accuracy_score,
)

import numpy as np

import torchvision

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

In [2]:
sys.path.append("../signal_processing/utils/")
from training_utils import (
    normalise_arrays,
    binarise,
    get_net_optimiser_scheduler_criterion,
    StabilityDiagrams,
)

from augmentations import augment_batch, sample_simple_augmentation_factors

# Setting a computing device for pytorch

In [3]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")
print(device)

cuda:4


# Loading data

See the notebook "Illustration of data" for more info on the data

In [4]:
root = "../data/"
nw_data_path = (
    root + "cross_architecture_tuning_data/20210819_20201031_processed_nanowire_data"
)
finfet_data_path = (
    root + "cross_architecture_tuning_data/20210915_20200715_processed_finFET_data.pkl"
)
heterostructure_data_path = (
    root + "cross_architecture_tuning_data/20210915_20200625_processed_sigeHet_data.pkl"
)

heterostructure_gaas_data_path = root + "faster_than_human_tuning_data/current_maps.npy"

nw_bt_data_path = root + "various_nice_nanowire_data/nanowires_cut_down_imgs.npy"

In [5]:
with open(nw_data_path, "rb") as f:
    nw_data = pickle.load(f)
with open(finfet_data_path, "rb") as f:
    finfet_data = pickle.load(f)
with open(heterostructure_data_path, "rb") as f:
    hs_data = pickle.load(f)


hs_gaas_data = np.load(heterostructure_gaas_data_path, allow_pickle=True)

nw_other_data = np.load(nw_bt_data_path, allow_pickle=True)

# We need to reshape some of the data so it is uniform

We choose 48 x 48 as our default image size

In [6]:
hs_gaas_data_resized = []
for ind, c in enumerate(hs_gaas_data):
    c = resize(c, (48, 48), anti_aliasing=True)
    hs_gaas_data_resized.append(c.copy())
hs_gaas_data = np.array(hs_gaas_data_resized)


nw_other_data_resized = []
for ind, c in enumerate(nw_other_data):
    c = resize(c, (48, 48), anti_aliasing=True)
    nw_other_data_resized.append(c.copy())
nw_other_data = np.array(nw_other_data_resized)

# Depending on the data source, we need to attach labels in different ways

In [7]:
X_nw = np.array(nw_data["high_res_scan"])
print("Shape of the X's", X_nw.shape)

y_nw = np.array(nw_data["label"])
print("Shape of the Y's", y_nw.shape)
X_finfet = np.array(finfet_data["high_res_scan"])
print("Shape of the X's", X_finfet.shape)

y_finfet = np.array(finfet_data["label"])
print("Shape of the Y's", y_finfet.shape)
X_hs = np.array(hs_data["high_res_scan"])
print("Shape of the X's", X_hs.shape)

y_hs = np.array(hs_data["label"])
print("Shape of the Y's", y_hs.shape)

X_hs_gaas = hs_gaas_data
print("Shape of the X's", X_hs_gaas.shape)

y_hs_gaas = np.load(
    root + "faster_than_human_tuning_data/labels.npy", allow_pickle=True
)
print("Shape of the Y's", y_hs_gaas.shape)


X_nw_other = nw_other_data
print("Shape of the X's", X_nw_other.shape)

y_nw_other = np.ones(len(nw_other_data))
print("Shape of the Y's", y_nw_other.shape)

Shape of the X's (1755, 48, 48)
Shape of the Y's (1755,)
Shape of the X's (157, 48, 48)
Shape of the Y's (157,)
Shape of the X's (637, 48, 48)
Shape of the Y's (637,)
Shape of the X's (2048, 48, 48)
Shape of the Y's (2048,)
Shape of the X's (14, 48, 48)
Shape of the Y's (14,)


In [8]:
X = np.concatenate((X_nw, X_finfet, X_hs, X_hs_gaas, X_nw_other))
print("Shape of the X's", X.shape)

y = np.concatenate((y_nw, y_finfet, y_hs, y_hs_gaas, y_nw_other))
print("Shape of the X's", y.shape)

Shape of the X's (4611, 48, 48)
Shape of the X's (4611,)


In [9]:
# Now for some training hyperparameters:

In [10]:
n_epochs = 100
batch_size = 128  # size of minibatches
n_repetitions = 5

n_imgs_per_epoch = 40000

print_every_n_epoch = 33
test_size = 0.1

save_directory = "training_results/20220610_high_res_dqd_classifier/"

# Let's run it 🚀

In [11]:
for rep in range(n_repetitions):
    print("This is rep", rep)

    """
    Data prep
    """

    # separate data into train and test datasetes
    x_train, x_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=rep
    )
    # Data preprocessing
    # Scale images to the [0, 1] range
    x_train = normalise_arrays(x_train)
    x_test = normalise_arrays(x_test)
    # Scale labels to represent two clases (0,1)
    y_train = binarise(y_train)
    y_test = binarise(y_test)
    # Make sure images have shape (1, 48, 48) for pytorch

    x_test = np.expand_dims(x_test, 1)
    print("x_train shape:", x_train.shape)
    print(x_train.shape[0], "train samples")
    print(x_test.shape[0], "test samples")

    y_test_this_rep = []

    fold = 0

    """
    Augmentation
    
    This might seem a bit involved but the reason is that the 
    augmentation function augments each image in an array
    exactly once. We therefore repeat the augmentation a couple of times, enough
    so that we have enough data to match the ```n_imgs_per_epoch``` 
    requirement.
    """
    n_augmentations = n_imgs_per_epoch // (len(x_train)) + 1

    # x_train_ = np.repeat(x_train, n_augmentations, axis=0)
    # y_train_ = np.repeat(y_train, n_augmentations, axis=0)

    print("augmenting the real data this many times: ", n_augmentations)
    print("# data in real training data", len(x_train))
    x_train_augmented = []
    y_train_augmented = []
    for n_aug in tqdm(range(n_augmentations)):
        x_train_temporary = augment_batch(
            x_train,
            sampling_func=sample_simple_augmentation_factors,
            n_workers=5,
            multiprocessing_on=True,
        )
        x_train_augmented.append(x_train_temporary)
        y_train_augmented.append(y_train)
    x_train = np.array(x_train_augmented)
    y_train = np.array(y_train_augmented)

    """
    Reshape so it fits to pytorch stuff
    """
    x_train = x_train.reshape((-1, 1, x_train.shape[-2], x_train.shape[-1]))
    y_train = y_train.reshape(-1)

    idx = np.random.permutation(len(x_train))
    x_train = x_train[idx]
    y_train = y_train[idx]

    x_train = x_train[:n_imgs_per_epoch]
    y_train = y_train[:n_imgs_per_epoch]

    print("# data in training data after augmentation", len(x_train))
    """
    Getting all parts ready for training
    """
    dataset = StabilityDiagrams(imgs=x_train, labels=y_train)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    class_weights = sklearn.utils.class_weight.compute_class_weight(
        "balanced", [0, 1], y_train
    )
    class_weights = torch.FloatTensor(class_weights).to(device)
    net, optimizer, scheduler, criterion = get_net_optimiser_scheduler_criterion(
        device, class_weights=class_weights
    )
    loss_history = []
    lr_history = []
    """
    Running the training
    """
    for epoch in tqdm(range(n_epochs)):
        for i_batch, sample_batched in enumerate(dataloader):
            X_train_minibatch = sample_batched["image"].to(device).float()
            y_train_minibatch = sample_batched["label"].to(device).long()
            optimizer.zero_grad()
            outputs = net(X_train_minibatch)
            loss = criterion(outputs, y_train_minibatch)
            loss.backward()
            optimizer.step()
        loss_history.append(loss.item())
        lr_history.append(optimizer.param_groups[0]["lr"])
        scheduler.step(loss.item())
        if epoch % print_every_n_epoch == print_every_n_epoch - 1:
            print("[%d] loss: %.7f" % (epoch + 1, loss.item()))
            print("learning rate now:", optimizer.param_groups[0]["lr"])
            net.eval()
            outputs = net(torch.FloatTensor(x_test).to(device))
            predicted = torch.max(outputs.data, 1).indices.detach().cpu()
            print(confusion_matrix(y_test, predicted, labels=[0, 1]))
            print(balanced_accuracy_score(y_test, predicted))
            net.train()
    path_networks = "saved_networks/"
    os.makedirs(save_directory + path_networks, exist_ok=True)
    torch.save(
        net.state_dict(),
        save_directory + path_networks + "rep_" + str(rep) + ".pth",
    )
    net.eval()
    outputs = net(torch.FloatTensor(x_test).to(device))
    predicted = torch.max(outputs.data, 1).indices.detach().cpu()
    print(confusion_matrix(y_test, predicted, labels=[0, 1]))
    print(balanced_accuracy_score(y_test, predicted))
    net.train()
    print("#" * 40)
    print("#" * 40)
    print("#" * 40)
    print("#" * 40)

This is rep 0
x_train shape: (4149, 48, 48)
4149 train samples
462 test samples
augmenting the real data this many times:  10
# data in real training data 4149


  0%|          | 0/10 [00:00<?, ?it/s]

# data in training data after augmentation 40000


/home/jonas/miniconda3/envs/torchenv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0, 1], y=[0 1 0 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

# data in training data after augmentation 40000


/home/jonas/miniconda3/envs/torchenv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0, 1], y=[0 0 0 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


  0%|          | 0/100 [00:00<?, ?it/s]

[33] loss: 0.0044864
learning rate now: 1e-05
[[376  14]
 [ 16  56]]
0.870940170940171
[66] loss: 0.0007958
learning rate now: 1.0000000000000004e-08
[[377  13]
 [ 16  56]]
0.8722222222222222
[99] loss: 0.0006873
learning rate now: 1.0000000000000004e-08
[[375  15]
 [ 15  57]]
0.8766025641025641
[[375  15]
 [ 16  56]]
0.8696581196581197
########################################
########################################
########################################
########################################


# Now run it with all data 

"testing" on unaugmented original data.

In [16]:
"""
Data prep
"""

# separate data into train and test datasetes
x_train, x_test, y_train, y_test = X, X, y, y
# Data preprocessing
# Scale images to the [0, 1] range
x_train = normalise_arrays(x_train)
x_test = normalise_arrays(x_test)
# Scale labels to represent two clases (0,1)
y_train = binarise(y_train)
y_test = binarise(y_test)
# Make sure images have shape (1, 48, 48) for pytorch

x_test = np.expand_dims(x_test, 1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


y_test_this_rep = []

"""
Augmentation

This might seem a bit involved but the reason is that the 
augmentation function augments each image in an array
exactly once. We therefore repeat the augmentation a couple of times, enough
so that we have enough data to match the ```n_imgs_per_epoch``` 
requirement.
"""
n_augmentations = n_imgs_per_epoch // (len(x_train)) + 1

# x_train_ = np.repeat(x_train, n_augmentations, axis=0)
# y_train_ = np.repeat(y_train, n_augmentations, axis=0)

print("augmenting the real data this many times: ", n_augmentations)
print("# data in real training data", len(x_train))
x_train_augmented = []
y_train_augmented = []
for n_aug in tqdm(range(n_augmentations)):
    x_train_temporary = augment_batch(
        x_train,
        sampling_func=sample_simple_augmentation_factors,
        n_workers=5,
        multiprocessing_on=True,
    )
    x_train_augmented.append(x_train_temporary)
    y_train_augmented.append(y_train)
x_train = np.array(x_train_augmented)
y_train = np.array(y_train_augmented)

"""
Reshape so it fits to pytorch stuff
"""
x_train = x_train.reshape((-1, 1, x_train.shape[-2], x_train.shape[-1]))
y_train = y_train.reshape(-1)

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

x_train = x_train[:n_imgs_per_epoch]
y_train = y_train[:n_imgs_per_epoch]

print("# data in training data after augmentation", len(x_train))
"""
Getting all parts ready for training
"""
dataset = StabilityDiagrams(imgs=x_train, labels=y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)
class_weights = sklearn.utils.class_weight.compute_class_weight(
    "balanced", [0, 1], y_train
)
class_weights = torch.FloatTensor(class_weights).to(device)
net, optimizer, scheduler, criterion = get_net_optimiser_scheduler_criterion(
    device, class_weights=class_weights
)
loss_history = []
lr_history = []
"""
Running the training
"""
for epoch in tqdm(range(n_epochs)):
    for i_batch, sample_batched in enumerate(dataloader):
        X_train_minibatch = sample_batched["image"].to(device).float()
        y_train_minibatch = sample_batched["label"].to(device).long()
        optimizer.zero_grad()
        outputs = net(X_train_minibatch)
        loss = criterion(outputs, y_train_minibatch)
        loss.backward()
        optimizer.step()
    loss_history.append(loss.item())
    lr_history.append(optimizer.param_groups[0]["lr"])
    scheduler.step(loss.item())
    if epoch % print_every_n_epoch == print_every_n_epoch - 1:
        print("[%d] loss: %.7f" % (epoch + 1, loss.item()))
        print("learning rate now:", optimizer.param_groups[0]["lr"])
        net.eval()
        outputs = net(torch.FloatTensor(x_test).to(device))
        predicted = torch.max(outputs.data, 1).indices.detach().cpu()
        print(confusion_matrix(y_test, predicted, labels=[0, 1]))
        print(balanced_accuracy_score(y_test, predicted))
        net.train()
path_networks = "saved_networks/"
os.makedirs(save_directory + path_networks, exist_ok=True)
torch.save(
    net.state_dict(),
    save_directory + path_networks + "all_data.pth",
)
net.eval()
outputs = net(torch.FloatTensor(x_test).to(device))
predicted = torch.max(outputs.data, 1).indices.detach().cpu()
print(confusion_matrix(y_test, predicted, labels=[0, 1]))
print(balanced_accuracy_score(y_test, predicted))
net.train()

x_train shape: (4611, 48, 48)
4611 train samples
4611 test samples
augmenting the real data this many times:  9
# data in real training data 4611


  0%|          | 0/9 [00:00<?, ?it/s]

# data in training data after augmentation 40000


/home/jonas/miniconda3/envs/torchenv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0, 1], y=[0 0 0 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


  0%|          | 0/100 [00:00<?, ?it/s]

[33] loss: 0.0378987
learning rate now: 1e-05
[[3744  141]
 [  81  645]]
0.9260681578863397
[66] loss: 0.0077767
learning rate now: 1.0000000000000004e-08
[[3754  131]
 [  91  635]]
0.9204681068317432
[99] loss: 0.1195969
learning rate now: 1.0000000000000004e-08
[[3742  143]
 [  83  643]]
0.9244333471606199
[[3758  127]
 [  91  635]]
0.9209829073465436


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  